# Evaluate the Prediction from the RetinaNet
## Imports

In [1]:
import json
import os

import warnings
warnings.filterwarnings("ignore")


## Calculate Intersections-over-Union

Categorize the Predictions in 'true positives', 'false positives' and 'false negatives'

In [2]:
def overlap(r1,r2):
    '''
    Overlapping rectangles overlap both horizontally & vertically
    '''
    
    #print(r1)
    #print(r2)
    
    r1_left   = r1[0]
    r1_right  = r1[2]
    r1_bottom = r1[3]
    r1_top    = r1[1]
    
    r2_left   = r2[0]
    r2_right  = r2[2]
    r2_bottom = r2[3]
    r2_top    = r2[1]
    
    overlaps = False
    
    dx = min(r1[2], r2[2]) - max(r1[0], r2[0])
    dy = min(r1[3], r2[3]) - max(r1[1], r2[1])
    
    if (dx >= 0) and (dy >= 0):
        overlaps = True
        
    return overlaps

def intersects_with_gt(detected, gt, iou_thres, idx):
    
    #print(idx)
    
    tp = 0
    fp = 0
    
    if len(gt) is 0:
        print('No Ground-Truth boxes')
        return
    
    #print('Detected:', len(detected))
    #print('Ground-Truth:', len(gt))
    
    for i, detected_box in enumerate(detected):        
        for gt_box in gt:
            
            #print('Detected:', detected_box)
            #print('Ground-Truth:', gt_box)
            if overlap(detected_box, gt_box):
                
                a = detected_box
                b = gt_box
                                
                # COORDINATES OF THE INTERSECTION BOX
                x1 = max(a[0], b[0])
                y1 = max(a[1], b[1])
                x2 = min(a[2], b[2])
                y2 = min(a[3], b[3])

                # AREA OF OVERLAP - Area where the boxes intersect
                width = (x2 - x1)
                height = (y2 - y1)
        
                area_overlap = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

                # COMBINED AREA
                area_a = (a[2] - a[0]) * (a[3] - a[1])
                area_b = (b[2] - b[0]) * (b[3] - b[1])
                area_combined = area_a + area_b - area_overlap
                
                #print('Area a: ', area_a)
                #print('Area b: ', area_b)
                #print('Area combined: ', area_combined)
                #print('Area overlap: ', area_overlap)

                # RATIO OF AREA OF OVERLAP OVER COMBINED AREA
                #iou = area_overlap / (area_combined+epsilon)
                iou = area_overlap / area_combined
                #iou = area_overlap / area_combined
                
                #print('iou: ', iou, " num: ", (id_num + i))
                
                #print('IOU: ', iou)
                
                if(iou >= iou_thres):
                    tp += 1
     
    fn = len(gt) - tp
    if fn < 0: 
        fn = 0
        
    fp = len(detected) - tp
    
    result = {}
    result['tp'] = tp
    result['fp'] = fp
    result['fn'] = fn
    
    return result

## Load the ground-truth data for the mating-events

In [3]:
def read_json(filepath):
    
    result_data = dict()
    
    with open(filepath) as json_file:
        train_dataset = json.load(json_file)
        
    ann = train_dataset['annotations']
    #print(ann)
    
    counter = 0
    
    for a in ann:
        
        counter += 1
        
        if a['category_id'] is 1:
            
            #print(a)
            
            idx = a['image_id']
            tmp_box = [float(x) for x in a['bbox']]
            
            x2 = tmp_box[0] + tmp_box[2]
            y2 = tmp_box[1] + tmp_box[3]
            
            bbox = [tmp_box[0], tmp_box[1], x2, y2]
        
            if idx in result_data:
            
                result_data[idx].append(bbox)
            else:
                
                tmp = []
                tmp.append(bbox)
                result_data[idx] = tmp
    
    #print('Iterated through: ', counter, " images.")
    print('done')
    return result_data
            

## Filter out the mating-events from the Predictions

In [4]:
def get_matings(pred_dataset):

    pred_data = dict()

    for d in pred_dataset:
    
        idx = d['image_id']
        bbox = [float(x) for x in d['bbox']]
        
        if d['category_id'] is 1:
    
            if idx in pred_data:
        
                pred_data[idx].append(bbox)        
            else:
        
                tmp = []
        
                tmp.append(bbox)
        
                pred_data[idx] = tmp

    print(len(pred_data), ' done')
    
    return pred_data

## Import Data for the Train and Validation dataset

In [164]:
ROOT_DIR = '/scratch/ganswindt/retinanet/COCO/DIR/'
ANNOTATION_DIR = os.path.join(ROOT_DIR, "annotations")

results_filepath = '/scratch/ganswindt/retinanet/'
val = os.path.join(results_filepath, 'result_1906-val-2.json')
train = os.path.join(results_filepath, 'result_1906-train-2.json')

In [165]:
with open(val) as json_file:
    dataset_val = json.load(json_file)

pred_data_val = get_matings(dataset_val)

with open(train) as json_file:
    dataset_train = json.load(json_file)

pred_data_train = get_matings(dataset_train)

8  done
71  done


## Load the ground-truth data for the Train and Validation dataset

In [166]:
val_file = ANNOTATION_DIR + '/instances_val.json'
train_file = ANNOTATION_DIR + '/instances_train.json'
    
gt_boxes_val = read_json(val_file)
gt_boxes_train = read_json(train_file)

#print(gt_boxes_val[70])
print(len(gt_boxes_val))
print(len(gt_boxes_train))

done
done
8
71


## Categorize the Predictions for the Validation dataset in TP, FP and FN

(TP: true positives, fp: false positives, fn: false negatives)

In [167]:
results_val = dict()

results_val['tp'] = 0
results_val['fp'] = 0
results_val['fn'] = 0

counter = 0
counter_pred = 0

for d in pred_data_val:
    for g in gt_boxes_val:
        
        if d is g:
               
            boxes = pred_data_val[d]
            gt = gt_boxes_val[g]
            
            counter += len(gt)
            counter_pred += len(boxes)
            
            tmp = intersects_with_gt(boxes, gt, 0.5, d)
            
            results_val['tp'] = results_val['tp'] + tmp['tp']
            results_val['fp'] = results_val['fp'] + tmp['fp']
            results_val['fn'] = results_val['fn'] + tmp['fn']
        else:
            continue

print('Num Ground-Truth Boxes in Val: ', counter)
print('Num predicted Boxes for Val: ', counter_pred)

Num Ground-Truth Boxes in Val:  64
Num predicted Boxes for Val:  61


## Calculate Accuracy for the Validation dataset

In [168]:
print('Results: ', results_val)

#1306: 19.49 %
print(results_val['tp']/(results_val['tp'] + results_val['fp']) * 100, '%')

Results:  {'tp': 50, 'fp': 11, 'fn': 17}
81.9672131147541 %


## Categorize the Predictions for the Train dataset in TP, FP and FN

(TP: true positives, fp: false positives, fn: false negatives)

In [169]:
results_train = dict()

results_train['tp'] = 0
results_train['fp'] = 0
results_train['fn'] = 0

counter = 0
counter_pred = 0

for d in pred_data_train:
    for g in gt_boxes_train:
        
        if d is g:
               
            boxes = pred_data_train[d]
            gt = gt_boxes_train[g]
            
            counter += len(gt)
            counter_pred += len(boxes)
            
            tmp = intersects_with_gt(boxes, gt, 0.5, d)
            
            results_train['tp'] = results_train['tp'] + tmp['tp']
            results_train['fp'] = results_train['fp'] + tmp['fp']
            results_train['fn'] = results_train['fn'] + tmp['fn']
        else:
            continue

print('Num Ground-Truth Boxes in Train: ', counter)
print('Num predicted Boxes for Val: ', counter_pred)

Num Ground-Truth Boxes in Train:  629
Num predicted Boxes for Val:  610


## Calculate Accuracy for the Train dataset

In [170]:
print('Results:', results_train)

#1306: 85.26 %
print(results_train['tp']/(results_train['tp'] + results_train['fp']) * 100, '%')

Results: {'tp': 591, 'fp': 19, 'fn': 102}
96.88524590163935 %


In [171]:
error_val = ((get_items(gt_boxes_val) - results_val['tp']) / get_items(gt_boxes_val)) * 100
error_train = ((get_items(gt_boxes_train) - results_train['tp']) / get_items(gt_boxes_train)) * 100

print('Error Val: ', error_val)
print('Error Train: ', error_train)

Error Val:  21.875
Error Train:  6.041335453100159


## With cls_tresh = 0.7:

In [12]:
if 1 == 0:
    results_filepath = '/scratch/ganswindt/retinanet/'
    val = os.path.join(results_filepath, 'result_1306-2.json')
    train = os.path.join(results_filepath, 'result_1306_train-2.json')

    with open(val) as json_file:
        dataset_val = json.load(json_file)

    pred_data_val = get_matings(dataset_val)

    with open(train) as json_file:
        dataset_train = json.load(json_file)

    pred_data_train = get_matings(dataset_train)

In [13]:
if 1 == 0:
    results_val = dict()

    results_val['tp'] = 0
    results_val['fp'] = 0
    results_val['fn'] = 0

    counter = 0
    counter_pred = 0

    for d in pred_data_val:
        for g in gt_boxes_val:

            if d is g:

                boxes = pred_data_val[d]
                gt = gt_boxes_val[g]

                counter += len(gt)
                counter_pred += len(boxes)

                tmp = intersects_with_gt(boxes, gt, 0.5, d)

                results_val['tp'] = results_val['tp'] + tmp['tp']
                results_val['fp'] = results_val['fp'] + tmp['fp']
                results_val['fn'] = results_val['fn'] + tmp['fn']
            else:
                continue

    print('Num Ground-Truth Boxes in Val: ', counter)
    print('Num predicted Boxes for Val: ', counter_pred)

In [14]:
if 1 == 0:
    print('Results: ', results_val)

    print((results_val['tp']/(results_val['tp'] + results_val['fp'])) * 100, '%')

In [15]:
if 1 == 0:
    results_train = dict()

    results_train['tp'] = 0
    results_train['fp'] = 0
    results_train['fn'] = 0

    counter = 0
    counter_pred = 0

    for d in pred_data_train:
        for g in gt_boxes_train:

            if d is g:

                boxes = pred_data_train[d]
                gt = gt_boxes_train[g]

                counter += len(gt)
                counter_pred += len(boxes)

                tmp = intersects_with_gt(boxes, gt, 0.5, d)

                results_train['tp'] = results_train['tp'] + tmp['tp']
                results_train['fp'] = results_train['fp'] + tmp['fp']
                results_train['fn'] = results_train['fn'] + tmp['fn']
            else:
                continue

    print('Num Ground-Truth Boxes in Train: ', counter)
    print('Num predicted Boxes for Val: ', counter_pred)

In [16]:
if 1 == 0:
    print('Results: ', results_train)

    print(results_train['tp']/counter_pred * 100, '%')

## Evaluate with Mating Events splitted in 'good' and 'bad'

In [98]:
ROOT_DIR_2 = '/scratch/ganswindt/retinanet/COCO_2_mating/DIR/'
ANNOTATION_DIR_2 = os.path.join(ROOT_DIR_2, "annotations")

results_filepath_2 = '/scratch/ganswindt/retinanet/'
val_2 = os.path.join(results_filepath_2, 'result_2606-val-2.json')
train_2 = os.path.join(results_filepath_2, 'result_2606-train-2.json')

In [99]:
with open(val_2) as json_file:
    dataset_val_2 = json.load(json_file)

pred_data_val_2 = get_matings(dataset_val_2)

with open(train_2) as json_file:
    dataset_train_2 = json.load(json_file)

pred_data_train_2 = get_matings(dataset_train_2)

8  done
63  done


In [100]:
val_file_2 = ANNOTATION_DIR_2 + '/instances_val.json'
train_file_2 = ANNOTATION_DIR_2 + '/instances_train.json'
    
gt_boxes_val_2 = read_json(val_file_2)
gt_boxes_train_2 = read_json(train_file_2)

#print(gt_boxes_val[70])
print(len(gt_boxes_val_2))
print(len(gt_boxes_train_2))

done
done
6
59


In [101]:
results_train_2 = dict()

results_train_2['tp'] = 0
results_train_2['fp'] = 0
results_train_2['fn'] = 0

counter = 0
counter_pred = 0

for d in pred_data_train_2:
    for g in gt_boxes_train_2:
        
        if d is g:
               
            boxes = pred_data_train_2[d]
            gt = gt_boxes_train_2[g]
            
            counter += len(gt)
            counter_pred += len(boxes)
            
            tmp = intersects_with_gt(boxes, gt, 0.5, d)
            
            results_train_2['tp'] = results_train_2['tp'] + tmp['tp']
            results_train_2['fp'] = results_train_2['fp'] + tmp['fp']
            results_train_2['fn'] = results_train_2['fn'] + tmp['fn']
        else:
            continue

print('Num Ground-Truth Boxes in Train: ', counter)
print('Num predicted Boxes for Train: ', counter_pred)

Num Ground-Truth Boxes in Train:  136
Num predicted Boxes for Train:  159


In [102]:
print('Results:', results_train_2)

#1306: 85.26 %
print(results_train_2['tp']/(results_train_2['tp'] + results_train_2['fp']) * 100, '%')
# for Pred data with 3 classes: 39.93808049535604 %

Results: {'tp': 141, 'fp': 18, 'fn': 8}
88.67924528301887 %


In [103]:
results_val_2 = dict()

results_val_2['tp'] = 0
results_val_2['fp'] = 0
results_val_2['fn'] = 0

counter = 0
counter_pred = 0

for d in pred_data_val_2:
    for g in gt_boxes_val_2:
        
        if d is g:
               
            boxes = pred_data_val_2[d]
            gt = gt_boxes_val_2[g]
            
            counter += len(gt)
            counter_pred += len(boxes)
            
            tmp = intersects_with_gt(boxes, gt, 0.5, d)
            
            results_val_2['tp'] = results_val_2['tp'] + tmp['tp']
            results_val_2['fp'] = results_val_2['fp'] + tmp['fp']
            results_val_2['fn'] = results_val_2['fn'] + tmp['fn']
        else:
            continue

print('Num Ground-Truth Boxes in Val: ', counter)
print('Num predicted Boxes for Val: ', counter_pred)

Num Ground-Truth Boxes in Val:  9
Num predicted Boxes for Val:  14


In [104]:
print('Results: ', results_val_2)

#1306: 19.49 %
print(results_val_2['tp']/(results_val_2['tp'] + results_val_2['fp']) * 100, '%')

# for pred data with three classes: 34.61538461538461 %

Results:  {'tp': 8, 'fp': 6, 'fn': 2}
57.14285714285714 %


## Calculate True Negatives

In [92]:
def get_all_no_matings(pred_dataset):

    pred_data = dict()

    for d in pred_dataset:
    
        idx = d['image_id']
        bbox = [float(x) for x in d['bbox']]
        
        if d['category_id'] is not 1:
    
            if idx in pred_data:
        
                pred_data[idx].append(bbox)        
            else:
        
                tmp = []
        
                tmp.append(bbox)
        
                pred_data[idx] = tmp

    print(len(pred_data), ' done')
    
    return pred_data

In [93]:
def read_all_json(filepath):
    
    result_data = dict()
    
    with open(filepath) as json_file:
        train_dataset = json.load(json_file)
        
    ann = train_dataset['annotations']
    #print(ann)
    
    counter = 0
    
    for a in ann:
        
        counter += 1
        
        if a['category_id'] is not 1:
            
            #print(a)
            
            idx = a['image_id']
            tmp_box = [float(x) for x in a['bbox']]
            
            x2 = tmp_box[0] + tmp_box[2]
            y2 = tmp_box[1] + tmp_box[3]
            
            bbox = [tmp_box[0], tmp_box[1], x2, y2]
        
            if idx in result_data:
            
                result_data[idx].append(bbox)
            else:
                
                tmp = []
                tmp.append(bbox)
                result_data[idx] = tmp
    
    #print('Iterated through: ', counter, " images.")
    print('done')
    return result_data

In [94]:
def get_items(li):
    
    count = 0
    for x in li:
        if isinstance(li[x], list): 
            count += len(li[x])
    
    return count

In [95]:
pred_no_val = get_all_no_matings(dataset_val)

pred_no_train = get_all_no_matings(dataset_train)

7  done
69  done


In [96]:
gt_all_val = read_all_json(val_file)
gt_all_train = read_all_json(train_file)

#print(gt_boxes_val[70])
print(len(gt_all_val))
print(len(gt_all_train))

done
done
8
72


In [50]:
tn_val = get_items(gt_all_val) - get_items(pred_no_val)
tn_train = get_items(gt_all_train) - get_items(pred_no_train)

results_val['tn'] = tn_val
results_train['tn'] = tn_train

In [51]:
print('Val: ', results_val)
print('Train: ', results_train)

Val:  {'tp': 30, 'fp': 8, 'fn': 34, 'tn': 285}
Train:  {'tp': 357, 'fp': 15, 'fn': 272, 'tn': 2104}


### For dataset with 4 classes:

In [52]:
pred_no_val_2 = get_all_no_matings(dataset_val_2)

pred_no_train_2 = get_all_no_matings(dataset_train_2)

8  done
72  done


In [53]:
gt_all_val_2 = read_all_json(val_file)
gt_all_train_2 = read_all_json(train_file)

#print(gt_boxes_val[70])
print(len(gt_all_val_2))
print(len(gt_all_train_2))

done
done
8
72


In [54]:
tn_val_2 = get_items(gt_all_val_2) - get_items(pred_no_val_2)
tn_train_2 = get_items(gt_all_train_2) - get_items(pred_no_train_2)

results_val_2['tn'] = tn_val_2
results_train_2['tn'] = tn_train_2

In [68]:
print('Val: ', results_val_2)
print('Train: ', results_train_2)

Val:  {'tp': 10, 'fp': 9, 'fn': 2}
Train:  {'tp': 187, 'fp': 19, 'fn': 5}


In [105]:
error_val_2 = ((get_items(gt_boxes_val_2) - results_val_2['tp']) / get_items(gt_boxes_val_2)) * 100
error_train_2 = ((get_items(gt_boxes_train_2) - results_train_2['tp']) / get_items(gt_boxes_train_2)) * 100

print('GT: ', get_items(gt_boxes_val_2))
print('Prediction: ', results_val_2['tp'])
print('Error Val: ', error_val_2)
print('Error Train: ', error_train_2)

GT:  9
Prediction:  8
Error Val:  11.11111111111111
Error Train:  -3.6764705882352944


# Calculate Values

In [113]:
error_val = ((get_items(gt_boxes_val) - results_val['tp']) / get_items(gt_boxes_val)) * 100
error_train = ((get_items(gt_boxes_train) - results_train['tp']) / get_items(gt_boxes_train)) * 100

Sens_val = results_val['tp'] / results_val['tp'] + results_val['fn']
Spec_val = results_val['tn'] / results_val['tn'] + results_val['fp']
Prec_val = results_val['tp'] / results_val['tp'] + results_val['fp']
TPR_val = results_val['tp'] / results_val['tp'] + results_val['fn']
FPR_val = results_val['fp'] / results_val['fp'] + results_val['tn']
TNR_val = results_val['tn'] / results_val['tn'] + results_val['fp']
FNR_val = results_val['fn'] / results_val['fn'] + results_val['tp']

Sens_train = results_train['tp'] / results_train['tp'] + results_train['fn']
Spec_train = results_train['tn'] / results_train['tn'] + results_train['fp']
Prec_train = results_train['tp'] / results_train['tp'] + results_train['fp']
TPR_train = results_train['tp'] / results_train['tp'] + results_train['fn']
FPR_train = results_train['fp'] / results_train['fp'] + results_train['tn']
TNR_train = results_train['tn'] / results_train['tn'] + results_train['fp']
FNR_train = results_train['fn'] / results_train['fn'] + results_train['tp']

KeyError: 'tn'

In [114]:
print('Error Val: ', error_val)
print('Error Train: ', error_train)

print('Sens Val: ', Sens_val)
print('Spec Val: ', Spec_val)
print('Prec Val: ', Prec_val)
print('TPR Val: ', TPR_val)
print('FPR Val: ', FPR_val)
print('TNR Val: ', TNR_val)
print('FNR Val: ', FNR_val)

print('Sens Train: ', Sens_train)
print('Spec Train: ', Spec_train)
print('Prec Train: ', Prec_train)
print('TPR Train: ', TPR_train)
print('FPR Train: ', FPR_train)
print('TNR Train: ', TNR_train)
print('FNR Train: ', FNR_train)

Error Val:  43.75
Error Train:  31.319554848966614
Sens Val:  29.0
Spec Val:  9.0
Prec Val:  9.0
TPR Val:  35.0
FPR Val:  286.0
TNR Val:  9.0
FNR Val:  31.0
Sens Train:  273.0
Spec Train:  16.0
Prec Train:  16.0
TPR Train:  273.0
FPR Train:  2105.0
TNR Train:  16.0
FNR Train:  358.0
